In [ ]:
"""

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

"""

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
安裝套件
"""

!pip install line-bot-sdk flask flask-ngrok

In [ ]:
"""
引用套件
"""

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import LineBotApi, WebhookHandler

# 引用無效簽章錯誤
from linebot.exceptions import InvalidSignatureError

In [ ]:
# 設定主程序 => 設定Server啟用細節
app = Flask(__name__, static_url_path = "/素材" , static_folder = "./素材/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("HPNN2U0LfdttfMl7b5yCkW/h9yzuxYOGW4NYDnJuBD4qXTY/UagvGJK3OKAnGwQwgA1tYG9lPNFt92N+j33osr3nQ59os0a/bD+usRNyUumvLs0IYnHTR6FkmhCAaQ8WkcKZq6a3AJlA4tfw9jxLtgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("e20a1f0d6418dd3bebb261a9fb9374ab")

In [ ]:

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""
製作一個訊息判斷器 
***** (用戶傳特定關鍵字訊息(此為傳送json檔名)進來後，會去找該檔名的json檔案，然後依據該json內，訊息內容的類型(type)來決定要回覆用戶什麼) *****

讀取指定的json檔案後，把json解析成不同格式的SendMessage
讀取檔案，把內容轉換成json
將json轉換成消息，接著放回array中，並把array傳出。

"""

# ***** 本章重點 *****


# 引用會用到的套件(影像地圖訊息、文字發送訊息、圖像發送訊息、位置發送訊息、影像發送訊息、貼圖發送訊息、音訊發送訊息)
from linebot.models import (
    ImagemapSendMessage, TextSendMessage, ImageSendMessage,
    LocationSendMessage, FlexSendMessage, VideoSendMessage,
    StickerSendMessage, AudioSendMessage
    )

from linebot.models.template import ButtonsTemplate, CarouselTemplate, ConfirmTemplate, ImageCarouselTemplate

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    # 開啟檔案，轉成json
    with open(fileName, encoding="utf8") as f:
        jsonArray = json.load(f)
    
    # 解析json
    """
    1. 先做一個空白的array
    2. 判斷消息，裡面的json的type欄位(確認訊息的種類是什麼)
    3. 依據type欄位轉換成xxxxxSendMessage物件
    4. append先前的空白array

    """

    returnArray = []
    for jsonObject in jsonArray:

        # 讀取每個json物件的["type"]欄位，並用來做判斷
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))

        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))

        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))

        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))

        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject)) 

        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  

        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))

        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  

        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))    


    # 回傳
    return returnArray


In [ ]:

# 引用套件
from linebot.models import MessageEvent, TextMessage

# 當用戶發來特殊關鍵字(此為json檔案的名稱)時，告訴前台小秘書該怎麼做，或是該把信息轉交給誰，然後回覆用戶(回覆的內容)
@handler.add(MessageEvent, message=TextMessage)
def process_text_message(event):

  # 先準備一個「回覆用戶的內容陣列」
  result_message_array =[]

  # 將用戶輸入的文字(關鍵字)，在本地端找相應的json檔案
  replyJsonPath = event.message.text + ".json"

  # 然後將相應的json檔案丟到上面的訊息判斷函式進行判斷，然後將函式運行的結果回傳給「回覆用戶的內容陣列」
  result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

  # 透過line_bot_api發送「回覆用戶的內容陣列」給用戶
  line_bot_api.reply_message(event.reply_token, result_message_array)


In [ ]:
# 主程序運行
app.run()